# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f473890c580>
├── 'a' --> tensor([[-0.1020,  0.2347,  0.3972],
│                   [ 1.3855, -0.7387,  1.4258]])
└── 'x' --> <FastTreeValue 0x7f473890ceb0>
    └── 'c' --> tensor([[-1.3142, -0.7945,  0.0425,  0.0593],
                        [-1.1762,  1.9423, -0.9816, -0.1268],
                        [-0.0992, -0.3811, -1.4791, -0.7830]])

In [4]:
t.a

tensor([[-0.1020,  0.2347,  0.3972],
        [ 1.3855, -0.7387,  1.4258]])

In [5]:
%timeit t.a

64.6 ns ± 0.0448 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f473890c580>
├── 'a' --> tensor([[ 0.4173,  0.2181,  1.1964],
│                   [ 1.4881, -1.3061, -0.9865]])
└── 'x' --> <FastTreeValue 0x7f473890ceb0>
    └── 'c' --> tensor([[-1.3142, -0.7945,  0.0425,  0.0593],
                        [-1.1762,  1.9423, -0.9816, -0.1268],
                        [-0.0992, -0.3811, -1.4791, -0.7830]])

In [7]:
%timeit t.a = new_value

67.4 ns ± 0.0453 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1020,  0.2347,  0.3972],
               [ 1.3855, -0.7387,  1.4258]]),
    x: Batch(
           c: tensor([[-1.3142, -0.7945,  0.0425,  0.0593],
                      [-1.1762,  1.9423, -0.9816, -0.1268],
                      [-0.0992, -0.3811, -1.4791, -0.7830]]),
       ),
)

In [10]:
b.a

tensor([[-0.1020,  0.2347,  0.3972],
        [ 1.3855, -0.7387,  1.4258]])

In [11]:
%timeit b.a

58.3 ns ± 0.0932 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0168, -0.0196,  1.2651],
               [ 0.2143,  0.5465, -1.4070]]),
    x: Batch(
           c: tensor([[-1.3142, -0.7945,  0.0425,  0.0593],
                      [-1.1762,  1.9423, -0.9816, -0.1268],
                      [-0.0992, -0.3811, -1.4791, -0.7830]]),
       ),
)

In [13]:
%timeit b.a = new_value

498 ns ± 6.54 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

818 ns ± 0.694 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 19.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 367 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 535 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f47388f8d90>
├── 'a' --> tensor([[[-0.1020,  0.2347,  0.3972],
│                    [ 1.3855, -0.7387,  1.4258]],
│           
│                   [[-0.1020,  0.2347,  0.3972],
│                    [ 1.3855, -0.7387,  1.4258]],
│           
│                   [[-0.1020,  0.2347,  0.3972],
│                    [ 1.3855, -0.7387,  1.4258]],
│           
│                   [[-0.1020,  0.2347,  0.3972],
│                    [ 1.3855, -0.7387,  1.4258]],
│           
│                   [[-0.1020,  0.2347,  0.3972],
│                    [ 1.3855, -0.7387,  1.4258]],
│           
│                   [[-0.1020,  0.2347,  0.3972],
│                    [ 1.3855, -0.7387,  1.4258]],
│           
│                   [[-0.1020,  0.2347,  0.3972],
│                    [ 1.3855, -0.7387,  1.4258]],
│           
│                   [[-0.1020,  0.2347,  0.3972],
│                    [ 1.3855, -0.7387,  1.4258]]])
└── 'x' --> <FastTreeValue 0x7f47388f8f70>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 114 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4683b69f40>
├── 'a' --> tensor([[-0.1020,  0.2347,  0.3972],
│                   [ 1.3855, -0.7387,  1.4258],
│                   [-0.1020,  0.2347,  0.3972],
│                   [ 1.3855, -0.7387,  1.4258],
│                   [-0.1020,  0.2347,  0.3972],
│                   [ 1.3855, -0.7387,  1.4258],
│                   [-0.1020,  0.2347,  0.3972],
│                   [ 1.3855, -0.7387,  1.4258],
│                   [-0.1020,  0.2347,  0.3972],
│                   [ 1.3855, -0.7387,  1.4258],
│                   [-0.1020,  0.2347,  0.3972],
│                   [ 1.3855, -0.7387,  1.4258],
│                   [-0.1020,  0.2347,  0.3972],
│                   [ 1.3855, -0.7387,  1.4258],
│                   [-0.1020,  0.2347,  0.3972],
│                   [ 1.3855, -0.7387,  1.4258]])
└── 'x' --> <FastTreeValue 0x7f473889fe80>
    └── 'c' --> tensor([[-1.3142, -0.7945,  0.0425,  0.0593],
                        [-1.1762,  1.9423, -0.9816, -0.1268],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 56.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.3142, -0.7945,  0.0425,  0.0593],
                       [-1.1762,  1.9423, -0.9816, -0.1268],
                       [-0.0992, -0.3811, -1.4791, -0.7830]],
              
                      [[-1.3142, -0.7945,  0.0425,  0.0593],
                       [-1.1762,  1.9423, -0.9816, -0.1268],
                       [-0.0992, -0.3811, -1.4791, -0.7830]],
              
                      [[-1.3142, -0.7945,  0.0425,  0.0593],
                       [-1.1762,  1.9423, -0.9816, -0.1268],
                       [-0.0992, -0.3811, -1.4791, -0.7830]],
              
                      [[-1.3142, -0.7945,  0.0425,  0.0593],
                       [-1.1762,  1.9423, -0.9816, -0.1268],
                       [-0.0992, -0.3811, -1.4791, -0.7830]],
              
                      [[-1.3142, -0.7945,  0.0425,  0.0593],
                       [-1.1762,  1.9423, -0.9816, -0.1268],
                       [-0.0992, -0.3811, -1.4791, -0.7830]],

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 432 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.3142, -0.7945,  0.0425,  0.0593],
                      [-1.1762,  1.9423, -0.9816, -0.1268],
                      [-0.0992, -0.3811, -1.4791, -0.7830],
                      [-1.3142, -0.7945,  0.0425,  0.0593],
                      [-1.1762,  1.9423, -0.9816, -0.1268],
                      [-0.0992, -0.3811, -1.4791, -0.7830],
                      [-1.3142, -0.7945,  0.0425,  0.0593],
                      [-1.1762,  1.9423, -0.9816, -0.1268],
                      [-0.0992, -0.3811, -1.4791, -0.7830],
                      [-1.3142, -0.7945,  0.0425,  0.0593],
                      [-1.1762,  1.9423, -0.9816, -0.1268],
                      [-0.0992, -0.3811, -1.4791, -0.7830],
                      [-1.3142, -0.7945,  0.0425,  0.0593],
                      [-1.1762,  1.9423, -0.9816, -0.1268],
                      [-0.0992, -0.3811, -1.4791, -0.7830],
                      [-1.3142, -0.7945,  0.0425,  0.0593],
                   

In [28]:
%timeit Batch.cat(batches)

147 µs ± 988 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

329 µs ± 2.22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
